In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## Task 1: Read in all the historical data and combine it into a single data structure (data frame,table, etc.).

In [87]:
#Reading all historical data
data_16 = pd.read_csv('../historicalPriceData/ERCOT_DA_Prices_2016.csv')
data_17 = pd.read_csv('../historicalPriceData/ERCOT_DA_Prices_2017.csv')
data_18 = pd.read_csv('../historicalPriceData/ERCOT_DA_Prices_2018.csv')
data_19 = pd.read_csv('../historicalPriceData/ERCOT_DA_Prices_2019.csv')

In [88]:
data_16.dtypes

Date                object
SettlementPoint     object
Price              float64
dtype: object

In [211]:
#Combining them into a signle dataframe
df = pd.concat([data_16, data_17, data_18, data_19]) #This dataframe is just in case I need the original later on
hp = pd.concat([data_16, data_17, data_18, data_19])

hp.head()

,Date,SettlementPoint,Price
0,2016-01-01 00:00:00,HB_BUSAVG,18.42
1,2016-01-01 00:00:00,HB_HOUSTON,18.42
2,2016-01-01 00:00:00,HB_HUBAVG,18.42
3,2016-01-01 00:00:00,HB_NORTH,18.43
4,2016-01-01 00:00:00,HB_SOUTH,18.38


In [143]:
#Checking for null values
hp.isnull().sum()

Date               0
SettlementPoint    0
Price              0
dtype: int64

In [144]:
#Checking for unique values that may be odd
sorted(list(hp['Date'].unique()), reverse=True)[0:3], sorted(list(hp['SettlementPoint'].unique()), reverse=True)[0:3], sorted(list(hp['Price'].unique()), reverse=True)[0:3]

(['2019-12-31 23:00:00', '2019-12-31 22:00:00', '2019-12-31 21:00:00'],
 ['LZ_WEST', 'LZ_SOUTH', 'LZ_RAYBN'],
 [5015.25, 5012.22, 5011.71])

In [145]:
# Change Date column to be datetime dtype
hp['Date'] = pd.to_datetime(hp['Date'])
hp.head()

,Date,SettlementPoint,Price
0,2016-01-01,HB_BUSAVG,18.42
1,2016-01-01,HB_HOUSTON,18.42
2,2016-01-01,HB_HUBAVG,18.42
3,2016-01-01,HB_NORTH,18.43
4,2016-01-01,HB_SOUTH,18.38


In [93]:
hp.dtypes

Date               datetime64[ns]
SettlementPoint            object
Price                     float64
dtype: object

## Task 2: Compute the average price for each settlement point and year-month in the historical dataset (48 year-months: January 2016 through December 2019).
- Compute these monthly average prices for all settlement points (both hubs and
    load zones) in the historical dataset. Hubs are denoted by the prefix "HB_" and
    load zones by the prefix “LZ_” in the SettlementPoint name.
- Do not filter out prices less than or equal to zero in the historical data when
    computing averages. Negative prices are indeed possible in deregulated power
    markets

In [94]:
hp['Year'] = hp['Date'].dt.year
hp['Month'] = hp['Date'].dt.month
hp.head()

,Date,SettlementPoint,Price,Year,Month
0,2016-01-01,HB_BUSAVG,18.42,2016,1
1,2016-01-01,HB_HOUSTON,18.42,2016,1
2,2016-01-01,HB_HUBAVG,18.42,2016,1
3,2016-01-01,HB_NORTH,18.43,2016,1
4,2016-01-01,HB_SOUTH,18.38,2016,1


In [95]:
hp.groupby(['Year','Month','SettlementPoint'])[['Price']].mean()

Price
Year Month SettlementPoint           
2016 1     HB_BUSAVG        19.209301
           HB_HOUSTON       19.565726
           HB_HUBAVG        19.258696
           HB_NORTH         19.133306
           HB_SOUTH         19.210054
...                               ...
2019 12    LZ_LCRA          19.456882
           LZ_NORTH         19.006801
           LZ_RAYBN         19.085067
           LZ_SOUTH         19.786882
           LZ_WEST          39.502594

[681 rows x 1 columns]

## Task 3: Write the computed monthly average prices to file as a CSV named AveragePriceByMonth.csv.
- The CSV file you write should have four columns with the following names:
    SettlementPoint, Year, Month, AveragePrice.

In [81]:
hp.groupby(['Year','Month','SettlementPoint'])[['Price']].mean()

#Saving as DateFrame
task_3 = hp.groupby(['Year','Month','SettlementPoint'])[['Price']].mean()

#Changing 'Price' column name to 'AveragePrice'
task_3.rename(columns={'Price': 'AveragePrice'}, inplace=True)
task_3

AveragePrice
Year Month SettlementPoint              
2016 1     HB_BUSAVG           19.209301
           HB_HOUSTON          19.565726
           HB_HUBAVG           19.258696
           HB_NORTH            19.133306
           HB_SOUTH            19.210054
...                                  ...
2019 12    LZ_LCRA             19.456882
           LZ_NORTH            19.006801
           LZ_RAYBN            19.085067
           LZ_SOUTH            19.786882
           LZ_WEST             39.502594

[681 rows x 1 columns]

In [83]:
#Saving it as a CSV file

task_3.to_csv('../output_hans/AveragePriceByMonth.csv')

## Task 4: Compute the hourly price1 volatility for each year and each settlement hub in the historical power price data (hubs are denoted by the prefix "HB_" in the SettlementPoint name).
- Since the natural logarithm is only defined for positive values, be sure to first filter
    out any prices that are zero or negative before computing log returns.
- Do not compute volatilities for the load zones in the data (prefix "LZ_" in the
    SettlementPoint name) for this question.

In [212]:
df.head()

,Date,SettlementPoint,Price
0,2016-01-01 00:00:00,HB_BUSAVG,18.42
1,2016-01-01 00:00:00,HB_HOUSTON,18.42
2,2016-01-01 00:00:00,HB_HUBAVG,18.42
3,2016-01-01 00:00:00,HB_NORTH,18.43
4,2016-01-01 00:00:00,HB_SOUTH,18.38


In [213]:
hp_2 = df
hp_2.shape

(497320, 3)

In [214]:
# indexes for negative and null values in Price column
index = hp_2[hp_2['Price'] <= 0].index

# Delete these row indexes from hp_2
hp_2.drop(index , inplace=True)
hp_2.shape

(493236, 3)

In [215]:
# computing log returns

hp_2['natural_log_price'] = np.log(hp_2['Price'])

#Dropping Price column
hp_2.drop(columns = 'Price', inplace = True)
hp_2.head()

,Date,SettlementPoint,natural_log_price
0,2016-01-01 00:00:00,HB_BUSAVG,2.913437
1,2016-01-01 00:00:00,HB_HOUSTON,2.913437
2,2016-01-01 00:00:00,HB_HUBAVG,2.913437
3,2016-01-01 00:00:00,HB_NORTH,2.913980
4,2016-01-01 00:00:00,HB_SOUTH,2.911263


In [216]:
hp_hb = hp_2[hp_2['SettlementPoint'].str.contains('LZ_') == False]

#checking for unique values in SettlementPoint value
hp_hb['SettlementPoint'].unique()

array(['HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_SOUTH',
       'HB_WEST', 'HB_PAN'], dtype=object)

In [217]:
hp_hb.head()

,Date,SettlementPoint,natural_log_price
0,2016-01-01 00:00:00,HB_BUSAVG,2.913437
1,2016-01-01 00:00:00,HB_HOUSTON,2.913437
2,2016-01-01 00:00:00,HB_HUBAVG,2.913437
3,2016-01-01 00:00:00,HB_NORTH,2.913980
4,2016-01-01 00:00:00,HB_SOUTH,2.911263


In [218]:
hp_hb.dtypes

Date                  object
SettlementPoint       object
natural_log_price    float64
dtype: object

In [219]:
hp_hb['Date'] = pd.to_datetime(hp_hb['Date'])
hp_hb.head()

<ipython-input-219-d906bb0f5a73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hp_hb['Date'] = pd.to_datetime(hp_hb['Date'])


,Date,SettlementPoint,natural_log_price
0,2016-01-01,HB_BUSAVG,2.913437
1,2016-01-01,HB_HOUSTON,2.913437
2,2016-01-01,HB_HUBAVG,2.913437
3,2016-01-01,HB_NORTH,2.913980
4,2016-01-01,HB_SOUTH,2.911263


In [220]:
hp_hb['Year'] = hp_hb['Date'].dt.year
hp_hb['Hour'] = hp_hb['Date'].dt.hour
hp_hb.head()

<ipython-input-220-d2b4baaa9cf2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hp_hb['Year'] = hp_hb['Date'].dt.year
<ipython-input-220-d2b4baaa9cf2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hp_hb['Hour'] = hp_hb['Date'].dt.hour


,Date,SettlementPoint,natural_log_price,Year,Hour
0,2016-01-01,HB_BUSAVG,2.913437,2016,0
1,2016-01-01,HB_HOUSTON,2.913437,2016,0
2,2016-01-01,HB_HUBAVG,2.913437,2016,0
3,2016-01-01,HB_NORTH,2.913980,2016,0
4,2016-01-01,HB_SOUTH,2.911263,2016,0


In [221]:
hp_hb.groupby(['Year','Hour','SettlementPoint'])[['natural_log_price']].std()

#Saving as DateFrame
task_4 = hp_hb.groupby(['Year','Hour','SettlementPoint'])[['natural_log_price']].std()

#Changing 'natural_log_price' column name to 'HourlyVolatility'
task_4.rename(columns={'natural_log_price': 'HourlyVolatility'}, inplace=True)
task_4

HourlyVolatility
Year Hour SettlementPoint                  
2016 0    HB_BUSAVG                0.403882
          HB_HOUSTON               0.397947
          HB_HUBAVG                0.420049
          HB_NORTH                 0.389115
          HB_SOUTH                 0.406438
...                                     ...
2019 23   HB_HUBAVG                0.235340
          HB_NORTH                 0.208033
          HB_PAN                   0.817654
          HB_SOUTH                 0.197773
          HB_WEST                  0.710928

[600 rows x 1 columns]

## Task 5: Write the computed hourly volatilities for each settlement hub and each historical year to file as a CSV named HourlyVolatilityByYear.csv.
- The CSV file you write should have three columns with the following names:
    SettlementPoint, Year, HourlyVolatility.
- Column names are case sensitive.

In [222]:
task_4.head()

HourlyVolatility
Year Hour SettlementPoint                  
2016 0    HB_BUSAVG                0.403882
          HB_HOUSTON               0.397947
          HB_HUBAVG                0.420049
          HB_NORTH                 0.389115
          HB_SOUTH                 0.406438

In [223]:
task_4.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 600 entries, (2016, 0, 'HB_BUSAVG') to (2019, 23, 'HB_WEST')
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   HourlyVolatility  600 non-null    float64
dtypes: float64(1)
memory usage: 6.9+ KB


In [224]:
task_4 = task_4.reset_index()
task_4.drop(columns = 'Hour', inplace = True)

In [225]:
#Saving it as a CSV file

task_4.to_csv('../output_hans/HourlyVolatilityByYear.csv', index = False)

# Task 6: Determine which settlement hub showed the highest overall hourly volatility for each historical year. Write code to extract the rows of your computed hourly volatility table corresponding to these maxima and write this table to a second output file named MaxVolatilityByYear.csv.
- Your file should contain the same column names as the file you wrote in Task 5.

In [197]:
task_6 = pd.read_csv('../output_hans/HourlyVolatilityByYear.csv', index_col=0)
task_6.head()

,Year,SettlementPoint,HourlyVolatility
0,2016,HB_BUSAVG,0.403882
1,2016,HB_HOUSTON,0.397947
2,2016,HB_HUBAVG,0.420049
3,2016,HB_NORTH,0.389115
4,2016,HB_SOUTH,0.406438


In [207]:
task_6.groupby(['Year'])['SettlementPoint', 'HourlyVolatility'].max()

<ipython-input-207-caa222390fbb>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_6.groupby(['Year'])['SettlementPoint', 'HourlyVolatility'].max()


,SettlementPoint,HourlyVolatility
Year,,
2016,HB_WEST,0.638195
2017,HB_WEST,0.804503
2018,HB_WEST,0.878491
2019,HB_WEST,1.604769


In [210]:
#Saving it as a CSV file
task_6 = task_6.groupby(['Year'])['SettlementPoint', 'HourlyVolatility'].max()
task_6.to_csv('../output_hans/MaxVolatilityByYear.csv')

<ipython-input-210-3d161feead0b>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_6 = task_6.groupby(['Year'])['SettlementPoint', 'HourlyVolatility'].max()


## Task 7: Use the files in the supplementalMaterials folder as examples of the required data format, write code to translate the power price data structure you created in Task 1 into a format that could be readily consumed by the cQuant price simulation models, and write the data to separate files for each settlement point.
- The historical data provided to you uses the hour-beginning convention.
- Each SettlementPoint should have its historical data saved to a separate file. That is, each CSV file you write should have data for one and only one SettlementPoint. There should be 15 files in total; one for each of the settlement points represented in the historical power price data.
- To keep things tidy, save these files in a subdirectory named formattedSpotHistory within your main output directory for the exercise.
- Use the convention "spot_<SettlementPointName>.csv" for the filenames
    of the data you write to file.

In [233]:
hp.head()

,Date,SettlementPoint,Price
0,2016-01-01 00:00:00,HB_BUSAVG,18.42
1,2016-01-01 00:00:00,HB_HOUSTON,18.42
2,2016-01-01 00:00:00,HB_HUBAVG,18.42
3,2016-01-01 00:00:00,HB_NORTH,18.43
4,2016-01-01 00:00:00,HB_SOUTH,18.38


In [234]:
# Change Date column to be datetime dtype
hp['Date'] = pd.to_datetime(hp['Date'])
hp.shape

(497320, 3)

In [246]:
# Get unique values to later loop through them
uniq = list(hp['SettlementPoint'].unique())
uniq

['HB_BUSAVG',
 'HB_HOUSTON',
 'HB_HUBAVG',
 'HB_NORTH',
 'HB_SOUTH',
 'HB_WEST',
 'LZ_AEN',
 'LZ_CPS',
 'LZ_HOUSTON',
 'LZ_LCRA',
 'LZ_NORTH',
 'LZ_RAYBN',
 'LZ_SOUTH',
 'LZ_WEST',
 'HB_PAN']

In [248]:
for i in uniq:
    m = hp[hp['SettlementPoint'].str.contains(i)]
    #Save csv file
    m.to_csv(f'../output_hans/formattedSpotHistory/spot_<{i}>.csv')